# Set Up Gemini API

In [1]:
!pip uninstall -qy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.5 MB/s eta 0:00:00


In [2]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

### Choose a model
**Best Choice: `gemini-1.5-pro-latest`**

Reason:

    - Strong NLP & Reasoning: Best for generating structured interview questions.
    
    - Handles Complex Tasks: Can evaluate candidate responses effectively.
    
    - Supports Multi-Turn Conversations: Can handle follow-ups dynamically.
    
    - Latest Optimizations: Ensures the best performance in Google AI’s Gemini Pro models.

- **Primary Model:** `gemini-1.5-pro-latest` for question generation & response evaluation.
- **Secondary Option:** `gemini-1.5-flash-latest` if speed is more critical than deep reasoning.
- **Embedding Model:** `text-embedding-004 if you` need vector-based response analysis.

In [5]:
from pprint import pprint
client = genai.Client(api_key=GOOGLE_API_KEY)

for model in client.models.list():
  if model.name == 'models/gemini-1.5-pro-latest':
    pprint(model.to_json_dict())
    break

{'description': 'Alias that points to the most recent production '
                '(non-experimental) release of Gemini 1.5 Pro, our mid-size '
                'multimodal model that supports up to 2 million tokens.',
 'display_name': 'Gemini 1.5 Pro Latest',
 'input_token_limit': 2000000,
 'name': 'models/gemini-1.5-pro-latest',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '001'}


# question generation


In [9]:
# Define the interview question generation prompt
prompt = """
You are an AI interviewer. Generate 5 interview questions for a software developer role 
focusing on Python, data structures, and problem-solving skills.
"""

# Make the API call
response = client.models.generate_content(
    model='gemini-1.5-pro-latest',
    contents=prompt)

# Print generated interview questions
print("\nInterview Questions:\n")
print(response.text)


Interview Questions:

1.  **Scenario:** Imagine you're building a system to track real-time stock prices.  You need to store and quickly retrieve the latest price for thousands of different stock symbols. Which Python data structure would you choose for this task and why?  Explain how you would use it, considering factors like lookup speed and memory efficiency. (Focuses on data structures, Python, and problem-solving related to efficient data retrieval)


2.  **Coding Challenge (describe the solution, don't necessarily write code):** You're given a log file containing timestamps and error messages. You need to write a Python function that identifies the most frequent error message within a specific time window (e.g., the last hour).  How would you approach this problem?  What data structures and algorithms would you use to optimize performance, especially with a very large log file?  (Focuses on problem-solving, algorithms, and data structures within a Python context)


3.  **Concept

# CV Parsing

In [11]:
!pip install pdfplumber docx2txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.7 MB/s eta 0:00:0000:01


In [12]:
cv_text = """John Doe
Software Engineer with 3 years of experience in AI and Computer Vision.
Skills: Python, OpenCV, TensorFlow, FastAPI.
Experience: Google - AI Engineer (2021-2024)"""

prompt = f"Extract structured information from this CV:\n{cv_text}"
response = client.models.generate_content(
    model='gemini-1.5-pro-latest',
    contents=prompt)

print(response.text)

```json
{
  "name": "John Doe",
  "title": "Software Engineer",
  "experience_summary": "3 years of experience in AI and Computer Vision",
  "skills": ["Python", "OpenCV", "TensorFlow", "FastAPI"],
  "experience": [
    {
      "company": "Google",
      "title": "AI Engineer",
      "years": "2021-2024"
    }
  ]
}
```



In [20]:
import pdfplumber
import docx2txt
import os

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Extract text from DOCX
def extract_text_from_docx(docx_path):
    return docx2txt.process(docx_path)

# Use Gemini API for CV parsing
def parse_cv_with_gemini(cv_text):
    prompt = f"""
    Extract key details from the following CV and return structured JSON output.
    CV Text: {cv_text}
    
    Return JSON with keys: "name", "email", "phone", "education", "experience", "skills".
    """
    response = client.models.generate_content(
        model='gemini-1.5-pro-latest',
        contents=prompt)
    return response.text  # Gemini returns a formatted JSON string

# Main function to process CV
def process_cv(file_path, file_type="pdf"):
    text = extract_text_from_pdf(file_path) if file_type == "pdf" else extract_text_from_docx(file_path)
    parsed_data = parse_cv_with_gemini(text)
    return parsed_data

# Example usage
cv_data = process_cv("/kaggle/input/resume-dataset/data/data/ENGINEERING/10624813.pdf", file_type="pdf")
print(cv_data)


```json
{
  "name": null,
  "email": null,
  "phone": null,
  "education": [
    {
      "degree": "Postgraduate courses",
      "major": "Cognitive and Neural Systems",
      "university": "Boston University",
      "year": 1991,
      "city": null,
      "state": null
    },
    {
      "degree": "M.S.",
      "major": "Computer Science",
      "university": "Union College",
      "year": 1989,
      "city": null,
      "state": null,
      "additional_info": "Masters Project: A Netless Neural Network - presented on August 16, 1989 at International Conference on Expert Systems and Neural Networks - Theory & Applications"
    },
    {
      "degree": "B.S.",
      "major": "Computer Science",
      "university": "Union College",
      "year": 1988,
      "city": null,
      "state": null
    }
  ],
  "experience": [
    {
      "title": "Engineering Operations Director",
      "company": "Current Company Name",
      "city": null,
      "state": null,
      "start_date": "January 2014

# Whisper Speech-to-Text (STT) Integration

In [24]:
! pip install openai ffmpeg pydub fastapi uvicorn
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.1 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.4 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803405 sha256=02ba077153840aa0d6e093a53d8a853d441df188ec6fdf8caba0980c18b974b3
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [25]:
import whisper
print("Whisper Version:", whisper.__version__)

Whisper Version: 20240930


In [37]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from pydub import AudioSegment
import numpy as np
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai
import time

# Load Whisper model locally
def load_whisper_model(model_size="base"):
    """
    Load Whisper model from Hugging Face transformers.
    Available sizes: tiny, base, small, medium, large
    Smaller models are faster but less accurate.
    """
    print(f"Loading Whisper {model_size} model...")
    processor = WhisperProcessor.from_pretrained(f"openai/whisper-{model_size}")
    model = WhisperForConditionalGeneration.from_pretrained(f"openai/whisper-{model_size}")
    
    # Move to GPU if available
    if torch.cuda.is_available():
        model = model.to("cuda")
        print("Using GPU for inference")
    else:
        print("Using CPU for inference")
    
    return model, processor

# Transcribe audio using local Whisper model
def transcribe_audio_local(audio_path, model_size="base"):
    # Load model
    model, processor = load_whisper_model(model_size)
    
    # Load and preprocess audio
    print("Converting audio...")
    audio = AudioSegment.from_file(audio_path)
    # Convert to mono and resample to 16kHz
    audio = audio.set_channels(1)
    audio = audio.set_frame_rate(16000)
    
    # Convert audio to numpy array
    audio_array = np.array(audio.get_array_of_samples()).astype(np.float32) / 32768.0
    
    # Process audio
    print("Transcribing...")
    start_time = time.time()
    
    input_features = processor(audio_array, sampling_rate=16000, return_tensors="pt").input_features
    if torch.cuda.is_available():
        input_features = input_features.to("cuda")
        
    # Generate tokens and then text
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    elapsed_time = time.time() - start_time
    print(f"Transcription completed in {elapsed_time:.2f} seconds")
    
    return transcription

# Process interview response using Gemini
def process_response_with_gemini(transcribed_text):
    prompt = f"Analyze this interview response: '{transcribed_text}'. Provide a structured JSON output with key insights."
    
    response = client.models.generate_content(
        model='gemini-1.5-pro',
        contents=prompt
    )
    return response.text

# Example usage
try:
    
    # Choose model size based on your needs:
    # - "tiny" or "base" for quick results (less accurate)
    # - "small" for balanced speed/accuracy
    # - "medium" or "large" for best accuracy (slower)
    model_size = "base"  # Change as needed
    
    transcribed_text = transcribe_audio_local(
        "/kaggle/input/sample-response/sample_interview.mp4", 
        model_size=model_size
    )
    print("\nTranscribed Text:", transcribed_text)
    
    ai_analysis = process_response_with_gemini(transcribed_text)
    print("\nAI Analysis:", ai_analysis)
except Exception as e:
    print(f"Error: {str(e)}")

Loading Whisper base model...
Using CPU for inference
Converting audio...
Transcribing...
Transcription completed in 2.89 seconds

Transcribed Text:  Hello, my name is Salpano Leed and I'm a fresh guide with from the College of Artificial Intelligence. I was measuring an intelligence system robotics and I have hands-on experience as I work on multiple projects during my academic journey.

AI Analysis: ```json
{
  "overall_impression": "Needs significant improvement. Grammatical errors and unclear phrasing detract from the message.",
  "strengths": {
    "enthusiasm": "Implied enthusiasm by mentioning 'multiple projects' and 'hands-on experience'.",
    "relevant_background": "Clearly states a background in AI and robotics."
  },
  "weaknesses": {
    "grammar_and_syntax": [
      "Incorrect preposition 'with from'",
      "Unclear phrasing 'measuring an intelligence system robotics'"
    ],
    "lack_of_specificity": [
      "Doesn't mention specific projects or technologies worked on.

# Sentiment Analysis for Candidate Responses (Bonus Task)

In [7]:
response_text = "I think I would be a good fit for the role, but I am still learning certain aspects."

prompt = f"Analyze the sentiment of this job interview response: '{response_text}'. Classify it as Positive, Neutral, or Negative and provide reasoning."

response = client.models.generate_content(
        model='gemini-1.5-pro-latest',
        contents=prompt)
print(response.text)


The sentiment is **Neutral**.

While the candidate expresses interest and a belief in their potential ("I think I would be a good fit for the role"), they also acknowledge a skill gap ("but I am still learning certain aspects").  The combination of a positive statement with a qualifying statement of a need for further development balances the overall tone to neutral.  They're not overly confident or negative, but realistically self-aware.



In [9]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:0000:01


In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
response_text = "I think I would be a good fit for the role, but I am still learning certain aspects."
sentiment_scores = analyzer.polarity_scores(response_text)

print(sentiment_scores)


{'neg': 0.0, 'neu': 0.703, 'pos': 0.297, 'compound': 0.6542}


# Implement Crew AI Orchestration\

In [ ]:
from crewai import Crew, Task, Agent
from langchain_google_genai import GoogleGenerativeAI


# Define Agents
cv_parser = Agent(
    role="CV Parser",
    goal="Extract key information from resumes",
    backstory="AI assistant specialized in HR resume processing",
    llm=gemini_llm
)

question_generator = Agent(
    role="Interview Question Generator",
    goal="Generate tailored interview questions based on the parsed CV",
    backstory="AI trained to generate relevant job interview questions",
    llm=gemini_llm
)

response_evaluator = Agent(
    role="Response Evaluator",
    goal="Analyze and score candidate responses",
    backstory="AI HR expert evaluating interview responses",
    llm=gemini_llm
)

# Define Tasks
cv_parsing_task = Task(
    description="Extract key details from resumes and structure them into JSON format.",
    agent=cv_parser
)

question_generation_task = Task(
    description="Generate tailored interview questions based on the extracted CV data.",
    agent=question_generator
)

evaluation_task = Task(
    description="Analyze candidate responses and score them based on HR metrics.",
    agent=response_evaluator
)

# Create Crew and Define Workflow
crew = Crew(
    agents=[cv_parser, question_generator, response_evaluator],
    tasks=[cv_parsing_task, question_generation_task, evaluation_task]
)

# Execute the Workflow
crew.kickoff()
